# Tracing with Research Agents

This notebook demonstrates the built-in tracing capabilities of the OpenAI Agents SDK using a real research agent with built-in tools. We'll explore how tracing automatically captures agent runs, tool calls, and create structured research reports.

## Key Concepts

- **Default Tracing**: Automatic tracking of agent runs and tool usage
- **Built-in Tools**: WebSearchTool for real web searches
- **Structured Outputs**: Using Pydantic models for formatted responses
- **Trace Organization**: Grouping related operations

View your traces at: [OpenAI Traces Dashboard](https://platform.openai.com/traces)

## Setup

In [32]:
import os
from pydantic import BaseModel
from typing import List
from agents import (
    Agent, 
    Runner, 
    RunConfig, 
    WebSearchTool, 
    trace, 
    custom_span
)

# Ensure OpenAI API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

## Part 1: Simple Research Agent with Tracing

Let's create a research agent using OpenAI's built-in WebSearchTool. Every tool call will be automatically traced.

In [33]:
# Create research agent with built-in web search tool
research_agent = Agent(
    name="Research Assistant",
    instructions="""You are a research assistant. Use web search to find current, 
    accurate information and provide well-researched answers with sources.""",
    model="gpt-4o",
    tools=[WebSearchTool()]
)

# This will automatically create a trace named "Agent workflow"
result = await Runner.run(
    research_agent,
    "What are the latest developments in artificial intelligence in 2024?"
)

print("Research Results:")
print(result.final_output)

Research Results:
In 2024, artificial intelligence (AI) experienced significant advancements across various domains, including technological innovations, regulatory frameworks, and international collaborations.

**Technological Innovations**

- **Multimodal AI Systems**: The development of multimodal AI systems, capable of processing and generating content across multiple formats such as text, audio, video, and images, marked a significant breakthrough. OpenAI's GPT-4o exemplified this trend, integrating various modalities seamlessly to perform tasks like generating written narratives from visual data and converting complex datasets into comprehensible visual representations. ([linkedin.com](https://www.linkedin.com/pulse/artificial-intelligence-2024-year-review-bob-cristello-gcx0e?utm_source=openai))

- **Generative AI in Creative Fields**: Generative AI tools, leveraging technologies like generative adversarial networks (GANs) and diffusion models, gained momentum in content creation

## Part 2: Custom Workflow Names and Metadata

In [34]:
# Research with custom trace metadata using OpenAI Agents SDK
from agents import trace
from datetime import datetime

with trace("Renewable Energy Research", metadata={
    "topic": "renewable_energy",
    "research_type": "trend_analysis",
    "date": datetime.now().strftime("%Y-%m-%d")
}):
    result = await Runner.run(
        research_agent,
        "What are the current trends in renewable energy technology?"
    )

print("Renewable Energy Research:")
print(result.final_output)

Renewable Energy Research:
As of August 2025, the renewable energy sector is experiencing significant advancements across various technologies, driven by innovation, policy shifts, and increasing global energy demands. Key trends include:

**1. Solar Energy Expansion**

- **Perovskite Solar Cells**: These next-generation cells have achieved efficiencies exceeding 25%, surpassing traditional silicon-based panels. Their flexibility and lightweight nature enable integration into diverse applications, from building materials to portable devices. ([ratedpower.com](https://ratedpower.com/blog/renewable-industry-developments-innovations-2025/?utm_source=openai))

- **Bifacial Solar Panels**: Capable of capturing sunlight on both sides, bifacial panels can generate up to 30% more electricity, especially in environments with reflective surfaces like snow or sand. This technology is becoming increasingly prevalent in new solar installations. ([425ppm.com](https://www.425ppm.com/en/blog/innovatio

## Part 3: Multi-Step Research with Higher-Level Traces

Group multiple research queries under a single trace for comprehensive analysis.

In [35]:
async def comprehensive_research():
    """Conduct multi-step research on a topic"""
    
    # Group all research under one trace
    with trace("Comprehensive AI Market Research"):
        
        # Step 1: Current market overview
        market_result = await Runner.run(
            research_agent,
            "What is the current size and growth of the AI market in 2024?"
        )
        
        # Step 2: Key players analysis
        players_result = await Runner.run(
            research_agent,
            "Who are the leading companies in AI development and what are their market positions?"
        )
        
        # Step 3: Future predictions
        future_result = await Runner.run(
            research_agent,
            "What are the predictions for AI market growth over the next 5 years?"
        )
        
        print("=== AI Market Overview ===")
        print(market_result.final_output[:300] + "...\n")
        
        print("=== Key Players ===")
        print(players_result.final_output[:300] + "...\n")
        
        print("=== Future Predictions ===")
        print(future_result.final_output[:300] + "...\n")
        
        return {
            "market_overview": market_result.final_output,
            "key_players": players_result.final_output,
            "future_predictions": future_result.final_output
        }

research_data = await comprehensive_research()

=== AI Market Overview ===
As of 2024, the global artificial intelligence (AI) market has experienced substantial growth, with various analyses providing insights into its size and projected expansion.

**Global Market Size and Growth:**

- According to Precedence Research, the global AI market was valued at approximately USD...

=== Key Players ===
As of August 27, 2025, several companies are at the forefront of artificial intelligence (AI) development, each holding significant positions in the market.

**1. NVIDIA Corporation**

NVIDIA has emerged as a pivotal player in the AI sector, primarily due to its advanced graphics processing units (G...

=== Future Predictions ===
The artificial intelligence (AI) market is poised for substantial growth over the next five years, with various segments and regions contributing to this expansion.

**Overall AI Market Growth:**

- **Global Market Size:** The AI market is projected to grow from $150.3 billion in 2023 to $460.89 bil...



## Part 4: Structured Research Reports with Tracing

Create agents that output structured data using Pydantic models. This shows how tracing works with structured outputs.

In [36]:
# --- OpenAI Agents SDK version ---
from agents import Agent, Runner, function_tool, RunConfig
from pydantic import BaseModel
from typing import List

# Define structured report format using Pydantic
class ResearchReport(BaseModel):
    title: str
    summary: str
    key_findings: List[str]
    sources: List[str]
    confidence_level: str  # "high", "medium", "low"
    date_researched: str

# Define a web search tool as a function tool for the agent

# Create agent with structured output using OpenAI Agents SDK
structured_research_agent = Agent(
    name="Structured Research Agent",
    instructions=(
        "Conduct thorough research using web search and create a structured report.\n\n"
        "Guidelines:\n"
        "- Use web search to find current, reliable information\n"
        "- Summarize findings clearly\n"
        "- List 3-5 key findings\n"
        "- Include actual source URLs when possible\n"
        "- Assess confidence level based on source quality and consistency"
    ),
    model="gpt-4o",
    tools=[WebSearchTool()],  # Use the function_tool-decorated function
    output_type=ResearchReport
)

# Generate structured report - this will be traced with structured output
structured_result = await Runner.run(
    structured_research_agent,
    "Research the impact of quantum computing on cybersecurity",
    run_config=RunConfig(  # <-- FIX: use run_config (not config)
        workflow_name="Quantum Computing Cybersecurity Report",
        trace_metadata={  # <-- FIX: use trace_metadata (not metadata)
            "report_type": "structured",
            "domain": "quantum_computing_cybersecurity"
        }
    )
)

# Access structured output
report = structured_result.final_output
print("=== STRUCTURED RESEARCH REPORT ===")
print(f"Title: {report.title}")
print(f"\nSummary: {report.summary}")
print("\nKey Findings:")
for i, finding in enumerate(report.key_findings, 1):
    print(f"{i}. {finding}")
print(f"\nConfidence Level: {report.confidence_level}")
print(f"Research Date: {report.date_researched}")
print("\nSources:")
for source in report.sources:
    print(f"- {source}")


=== STRUCTURED RESEARCH REPORT ===
Title: Impact of Quantum Computing on Cybersecurity

Summary: Quantum computing poses a significant threat to current cybersecurity practices by potentially rendering existing encryption methods obsolete. As quantum computers advance, they could decrypt sensitive data protected by algorithms like RSA and ECC, necessitating the development and adoption of post-quantum cryptography to ensure data security.

Key Findings:
1. Quantum computers can break widely used encryption methods such as RSA and ECC, compromising data security.
2. 'Harvest now, decrypt later' attacks involve collecting encrypted data now to decrypt once quantum capabilities are available.
3. Post-quantum cryptography (PQC) is being developed to create algorithms resistant to quantum attacks.
4. Organizations are urged to begin transitioning to PQC to maintain cyber resilience.
5. Government initiatives, like the USA Quantum Computing Cybersecurity Preparedness Act, are promoting the s

In [37]:
# Access structured output
report = structured_result.final_output

# Create nicely formatted markdown report
markdown_report = f"""# Structured Research Report

# Title: {report.title}

## Summary
{report.summary}

## Key Findings
""" + "\n".join([f"{i}. {finding}" for i, finding in enumerate(report.key_findings, 1)]) + f"""

**Confidence Level:** {report.confidence_level}  

**Research Date:** {report.date_researched}

## Sources
""" + "\n".join([f"- {source}" for source in report.sources]) + "\n"

# Save to file
with open("structured_report.md", "w", encoding="utf-8") as f:
    f.write(markdown_report)

# Print to terminal
print(markdown_report)


# Structured Research Report

# Title: Impact of Quantum Computing on Cybersecurity

## Summary
Quantum computing poses a significant threat to current cybersecurity practices by potentially rendering existing encryption methods obsolete. As quantum computers advance, they could decrypt sensitive data protected by algorithms like RSA and ECC, necessitating the development and adoption of post-quantum cryptography to ensure data security.

## Key Findings
1. Quantum computers can break widely used encryption methods such as RSA and ECC, compromising data security.
2. 'Harvest now, decrypt later' attacks involve collecting encrypted data now to decrypt once quantum capabilities are available.
3. Post-quantum cryptography (PQC) is being developed to create algorithms resistant to quantum attacks.
4. Organizations are urged to begin transitioning to PQC to maintain cyber resilience.
5. Government initiatives, like the USA Quantum Computing Cybersecurity Preparedness Act, are promoting the 

## Part 5: Research Pipeline with Custom Spans

Add custom instrumentation to track different phases of the research process.

In [38]:
from agents import Runner, trace
from agents.tracing import custom_span

async def research_pipeline(topic: str):
    """Complete research pipeline with custom spans"""
    
    with trace("Complete Research Pipeline"):
        
        # Phase 1: Initial research
        with custom_span("initial_research", data={"topic": topic}):  # FIX: use data= (not metadata=)
            print(f"🔍 Starting initial research on: {topic}")
            initial_result = await Runner.run(
                research_agent,
                f"Provide an overview of {topic} with current information"
            )
            print("✅ Initial research complete")
        
        # Phase 2: Deep dive analysis
        with custom_span("deep_analysis", data={"phase": "detailed_research"}):  # FIX
            print("🔬 Conducting detailed analysis...")
            detailed_result = await Runner.run(
                research_agent,
                f"What are the latest technical developments and challenges in {topic}?"
            )
            print("✅ Detailed analysis complete")
        
        # Phase 3: Generate structured report
        with custom_span("report_generation", data={"output_type": "structured"}):  # FIX
            print("📄 Generating structured report...")
            
            # Combine findings for final report
            combined_research = f"""
            Based on the following research findings, create a comprehensive report:
            
            Initial Overview:
            {initial_result.final_output}
            
            Detailed Analysis:
            {detailed_result.final_output}
            
            Topic: {topic}
            """
            
            final_report = await Runner.run(
                structured_research_agent,
                combined_research
            )
            print("✅ Report generation complete")
        
        return final_report.final_output

# Run the complete research pipeline
topic = "electric vehicle battery technology"
final_report = await research_pipeline(topic)

print("\n" + "="*60)
print("FINAL RESEARCH REPORT")
print("="*60)
print(f"Title: {final_report.title}")
print(f"\nSummary: {final_report.summary}")
print("\nKey Findings:")
for i, finding in enumerate(final_report.key_findings, 1):
    print(f"{i}. {finding}")

🔍 Starting initial research on: electric vehicle battery technology
✅ Initial research complete
🔬 Conducting detailed analysis...
✅ Detailed analysis complete
📄 Generating structured report...
✅ Report generation complete

FINAL RESEARCH REPORT
Title: Advancements and Challenges in Electric Vehicle Battery Technology

Summary: The landscape of electric vehicle (EV) battery technology is rapidly evolving with innovations focusing on improving energy density, cost efficiency, and safety. Current advancements include solid-state, sodium-ion, and lithium-sulfur batteries, alongside innovative performance improvements like fast-charging capabilities and advanced electrolytes. However, challenges such as material dependencies, recycling inefficiencies, and manufacturing scalability remain prevalent.

Key Findings:
1. Lithium-ion batteries remain the dominant choice for EVs, with variants like NMC and LFP offering trade-offs between cost, safety, and energy density.
2. Emerging technologies l

## Part 6: Tracing Controls

Control tracing behavior and sensitive data handling.

In [31]:
# Example 1: Research without tracing (for sensitive topics)
sensitive_result = await Runner.run(
    research_agent,
    "General information about data privacy regulations",
    run_config=RunConfig(
        tracing_disabled=True,  # No trace will be created
        workflow_name="Sensitive Research"
    )
)

print("Research without tracing completed")
print(sensitive_result.final_output[:200] + "...")

print("\n" + "-"*50 + "\n")

# Example 2: Research with metadata but no sensitive data in traces
result_no_sensitive = await Runner.run(
    research_agent,
    "What are best practices for API security?",
    run_config=RunConfig(
        workflow_name="API Security Research",
        trace_include_sensitive_data=False,  # Exclude sensitive data from traces
        trace_metadata={
            "security_research": True,
            "classification": "standard"
        }
    )
)

print("Secure research completed")
print(result_no_sensitive.final_output[:200] + "...")

Research without tracing completed
Data privacy regulations are laws designed to protect individuals' personal data and ensure its proper management by organizations. Some key regulations include:

1. **General Data Protection Regulati...

--------------------------------------------------

Secure research completed
Ensuring the security of APIs is crucial given the increasing reliance on them for connecting applications and services. Here are some best practices for API security:

1. **Authentication and Authori...


## Summary

### What We Demonstrated

1. **Real Research Agent**: Used OpenAI's WebSearchTool for actual web searches
2. **Automatic Tracing**: All tool calls and agent runs were automatically traced
3. **Structured Outputs**: Created formatted reports using Pydantic models
4. **Trace Organization**: Grouped related operations with `trace()` context managers
5. **Custom Spans**: Added instrumentation for different research phases
6. **Tracing Controls**: Demonstrated disabling tracing and sensitive data handling

### Key Benefits of Tracing

- **Debugging**: See exactly what searches were performed and their results
- **Performance**: Monitor how long each research phase takes
- **Audit Trail**: Track all research activities for compliance
- **Optimization**: Identify bottlenecks in your research workflows

### Best Practices

- Use descriptive workflow names for easy identification
- Add relevant metadata for filtering and analysis
- Group related operations with `trace()` contexts
- Use `trace_include_sensitive_data=False` for production
- Add custom spans for important business logic phases

### Viewing Your Traces

Visit [OpenAI Traces Dashboard](https://platform.openai.com/traces) to:
- Visualize your research workflows
- Debug tool call issues
- Monitor performance metrics
- Analyze usage patterns